In [1]:
import tensorflow as tf
import numpy as np
from trade_sim import Market
import tensorflow.contrib as contrib
import os

In [54]:
class Agent:
    def __init__(self, numberOfCurrencies, timeFrame, sess, initialPortfolio=1000.0):
        self._s = sess
        self.inputT = tf.placeholder(shape=[None, numberOfCurrencies, timeFrame, 3], dtype=tf.float32)
        self.conv1 = tf.layers.conv2d(inputs=self.inputT, filters=50, kernel_size=[1,3])
      #  self.conv1 = tf.nn.depthwise_conv2d(self.inputT, [1,3,3,4], [1,1,1,1], 'SAME')
        self.conv2 = tf.layers.conv2d(inputs=self.conv1, filters=50, kernel_size=[1,48])
        self.conv3 = tf.layers.conv2d(inputs=self.conv2, filters=1, kernel_size=[1,1])
        self.final = tf.layers.dense(self.conv3, 1)
        self._allocate = tf.nn.softmax(self.final, axis=1)
        
        self.priceChanges = tf.placeholder(shape=[None, numberOfCurrencies, 1], dtype=tf.float32)
        
        self.loss = tf.subtract(tf.constant(initialPortfolio), tf.matmul(tf.matrix_transpose(self.priceChanges), tf.scalar_mul(tf.constant(initialPortfolio), tf.reshape(self._allocate, [-1, numberOfCurrencies, 1]))))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        self._train = self.optimizer.minimize(self.loss)
        
    def act(self, observation):
        return self._s.run(self._allocate, feed_dict={self.inputT: observation})
    
    def train_step(self, obs, prices):
        batch_feed = {self.inputT : obs,
                     self.priceChanges: prices
                     }
        self._s.run(self._train, feed_dict=batch_feed)

In [55]:
def main():
    testSim = Market(['EUR','USD'], os.path.abspath('../Data_Processing/ProcessedData'))
    with tf.Session() as sess:
        test1 = Agent(len(testSim.currencies), 50, sess)
        sess.run(tf.global_variables_initializer())
        obs = testSim.processTimePeriod(50, testSim.getCurrentTime())
        
        prices = testSim.getRates(testSim.getCurrentTime())
#         print(obs)
#         print(testSim.df['EURUSD'].loc[0:60, 'Open'])
        print(test1.act([obs]))
        for _ in range(20000):
            test1.train_step([obs], [prices])
        print(test1.act([obs]))
        print(prices)
        print(test1._s.run(test1.loss, feed_dict={test1.inputT: [obs], test1.priceChanges:[prices]}))
        

In [56]:
if __name__ == "__main__":
    main()

[[[[0.49996093]]

  [[0.5000391 ]]]]
[[[[9.997080e-01]]

  [[2.919395e-04]]]]
[[1.        ]
 [0.99998098]]
[[[6.1035156e-05]]]
